##### import pandas as pd
import numpy as np
import re
import nltk
import csv
from IPython.display import Image
from IPython.display import display

In [2]:
import pandas as pd
disease = pd.read_csv('Final_FB_2804_utf16.csv', encoding='UTF-16LE')
display(disease.head(3))

/Users/chiachienhung/anaconda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (18) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,id.x,language,key,likes_count.x,from_id.x,from_name.x,message.x,created_time.x,type,link,...,sad_count,angry_count,join_id,from_id.y,from_name.y,message.y,created_time.y,likes_count.y,comments_count.y,id.y
0,474591366067852_479391832254472,NaN,bristol myers,26.0,4.745914e+14,Bristol-Myers Squibb,NaN,2014-12-31 20:00:00,photo,https://www.facebook.com/BristolMyersSquibb/ph...,...,0.0,0.0,479391832254472,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,474591366067852_479697292223926,NaN,bristol myers,119.0,4.745914e+14,Bristol-Myers Squibb,NaN,2014-12-31 20:00:00,photo,https://www.facebook.com/BristolMyersSquibb/ph...,...,0.0,0.0,479697292223926,1.015417e+16,Joanie Davidson,I was out there last week and it is so impress...,2016-05-03 11:54:13,0.0,0.0,479697292223926_480292432164412
2,474591366067852_479697292223926,NaN,bristol myers,119.0,4.745914e+14,Bristol-Myers Squibb,NaN,2014-12-31 20:00:00,photo,https://www.facebook.com/BristolMyersSquibb/ph...,...,0.0,0.0,479697292223926,1.020636e+16,Kevin Giordano,Fantastic!,2016-05-01 18:09:14,0.0,0.0,479697292223926_479739125553076


In [3]:
df = pd.DataFrame(disease, columns = ['key','id.x','created_time.x', 'message.x','created_time.y', 'message.y'])
rm_duplicates_x = df.drop_duplicates(subset=['key','message.x'])
rm_duplicates_y = df.drop_duplicates(subset=['key','message.y'])
#print(len(df),len(rm_duplicates_x),len(rm_duplicates_y))
#display(rm_duplicates_x.head(10))
#display(rm_duplicates_y.head(10))
message_x=rm_duplicates_x[['key', 'id.x', 'created_time.x', 'message.x']]
message_y=rm_duplicates_y[['key', 'id.x', 'created_time.y', 'message.y']]
message_x.columns=['key', 'id.x', 'created_time','message']
message_y.columns=['key', 'id.x', 'created_time','message']
#display(message_x.head(10))
#display(message_y.head(10))
#print(len(message_x),len(message_y))
frames = [message_x, message_y]
result = pd.concat(frames, keys=['message.x', 'message.y'])
display(result.head(3))
print(len(df),len(result))
result.to_csv("post_comment_utf16.csv", encoding='UTF-16LE',columns = ['key','id.x','created_time', 'message'])

key                             id.x  \
message.x 0  bristol myers  474591366067852_479391832254472   
          6  bristol myers  474591366067852_479091638951158   
          7  bristol myers  474591366067852_479814438878878   

                    created_time  \
message.x 0  2014-12-31 20:00:00   
          6  2016-04-29 17:02:13   
          7  2016-05-01 23:58:47   

                                                       message  
message.x 0                                                NaN  
          6  People and patients are at the center of every...  
          7  BMS is an example for other companies to emulate.

165531 167794


In [4]:
post_comment = pd.read_csv('post_comment_utf16.csv', encoding='UTF-16LE') 
#df.apply(lambda x: pd.lib.infer_dtype(x.values))
post_comment.columns=['xy','docid','key', 'id.x', 'created_time','message']
display(post_comment.head(3))
display(len(post_comment['key']))

,xy,docid,key,id.x,created_time,message
0,message.x,0,bristol myers,474591366067852_479391832254472,2014-12-31 20:00:00,NaN
1,message.x,6,bristol myers,474591366067852_479091638951158,2016-04-29 17:02:13,People and patients are at the center of every...
2,message.x,7,bristol myers,474591366067852_479814438878878,2016-05-01 23:58:47,BMS is an example for other companies to emulate.


167794

In [5]:
rm_na = post_comment.dropna()
dtime = rm_na.sort(['created_time'])
dfinal = pd.DataFrame(dtime, columns = ['xy','docid','key','id.x', 'created_time','message'])
dfinal.to_csv("post_comment_rm_utf16.csv", encoding='UTF-16LE',columns = ['xy','docid','key','id.x', 'created_time','message'])

/Users/chiachienhung/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:2: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  from ipykernel import kernelapp as app


In [6]:
dstart = pd.read_csv('post_comment_rm_utf16.csv', encoding='UTF-16LE')
display(dstart.head(3))
print(len(dstart['key']))

,Unnamed: 0,xy,docid,key,id.x,created_time,message
0,40593,message.x,136529,psoriasis,105226382844778_320199574680790,2012-01-01 00:24:50,Ahora s. Feliz ao y mucha salud para tod@s!!!
1,144772,message.y,136529,psoriasis,105226382844778_320199574680790,2012-01-01 00:26:16,Igualmente para todos!!!!!!
2,144773,message.y,136530,psoriasis,105226382844778_320199574680790,2012-01-01 00:29:53,Ms piel y menos placas!!!!!!


167425


In [7]:
from nltk import wordpunct_tokenize
from nltk.corpus import stopwords
def _calculate_languages_ratios(text):
    languages_ratios = {}
    tokens = wordpunct_tokenize(text)
    words = [word.lower() for word in tokens]
    for language in stopwords.fileids():
        stopwords_set = set(stopwords.words(language))
        words_set = set(words)
        common_elements = words_set.intersection(stopwords_set)

        languages_ratios[language] = len(common_elements) # language "score"

    return languages_ratios
def detect_language(text):
    ratios = _calculate_languages_ratios(text)
    most_rated_language = max(ratios, key=ratios.get)
    return most_rated_language

In [8]:
import pandas as pd
import numpy as np
import re
import nltk
import csv
nltk.download('stopwords')
from nltk.corpus import stopwords
#pip install langdetect
from langdetect import detect

df_fb = pd.read_csv('post_comment_rm_utf16.csv', encoding = 'utf-16LE', sep=',')
                         
with open('post_comment_preprocessing_utf16.csv', 'w', encoding='UTF-16LE', newline='') as csvfile:
    column = [['no','xy','docid','key','id.x','created_time','message','lang','re_message']]
    writer = csv.writer(csvfile)
    writer.writerows(column)
    
for i in range(len(df_fb['message'])):    
    features = []
    features.append(i)
    features.append(df_fb['xy'][i])
    features.append(df_fb['docid'][i])
    features.append(df_fb['key'][i])
    features.append(df_fb['id.x'][i])
    features.append(df_fb['created_time'][i])
    features.append(df_fb['message'][i])
    if(str(df_fb['message'][i])=="nan"):
        features.append('english')
        features.append(df_fb['message'][i])
    else:
        #tokens = nltk.word_tokenize(str(df_twitter['message'][i]))
        tokens=' '.join(re.findall(r"[\w']+", str(df_fb['message'][i]))).lower().split()
        postag=nltk.pos_tag(tokens)
        irlist=[',','.',':','#',';','CD','WRB','RB','PRP','...',')','(','-','``','@']
        wordlist=['co', 'https', 'http','rt','www','ve','don',"i'm","it's"]
        adjandn = [word for word,pos in postag if pos not in irlist and word not in wordlist and len(word)>2]
        #adjandn = [word for word,pos in postag if pos not in irlist]
        lang=detect_language(df_fb['message'][i])
        features.append(lang)
        #print(i,lang)
        stop = set(stopwords.words(lang))
        wordlist = [i for i in adjandn if i not in stop]
        features.append(' '.join(wordlist))
    with open('post_comment_preprocessing_utf16.csv', 'a', encoding='UTF-16LE', newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerows([features])

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/chiachienhung/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [9]:
df_postncomment = pd.read_csv('post_comment_preprocessing_utf16.csv', encoding = 'UTF-16LE', sep=',')
#display(df_postncomment.head(5))
df_english= df_postncomment.loc[df_postncomment['lang'] == 'english']
df_rm = df_english.drop_duplicates(subset=['key','re_message'])
display(df_rm.head(3))
print(len(df_postncomment['key']),len(df_rm['key']))
rm_english_na = df_rm.dropna()
#print(len(rm_english_na))
dfinal_fb = pd.DataFrame(rm_english_na, columns = ['no','xy','docid','key','id.x','created_time','message','lang','re_message'])
dfinal_fb.to_csv("final_preprocessing_utf16.csv", encoding='UTF-16LE',columns = ['no','xy','docid','key','id.x','created_time','message','lang','re_message'])

,no,xy,docid,key,id.x,created_time,message,lang,re_message
0,0,message.x,136529,psoriasis,105226382844778_320199574680790,2012-01-01 00:24:50,Ahora s. Feliz ao y mucha salud para tod@s!!!,english,ahora feliz mucha salud para tod
2,2,message.y,136530,psoriasis,105226382844778_320199574680790,2012-01-01 00:29:53,Ms piel y menos placas!!!!!!,english,piel menos placas
6,6,message.y,136534,psoriasis,105226382844778_320199574680790,2012-01-01 00:41:29,"muchas gracias, adios picores y hola nueva pie...",english,muchas gracias adios picores hola nueva piel


167425 116151


In [10]:
df_postn = pd.read_csv('final_preprocessing_utf16.csv', encoding = 'UTF-16LE', sep=',')
display(df_postn.head(3))
print(len(df_postn))

,Unnamed: 0,no,xy,docid,key,id.x,created_time,message,lang,re_message
0,0,0,message.x,136529,psoriasis,105226382844778_320199574680790,2012-01-01 00:24:50,Ahora s. Feliz ao y mucha salud para tod@s!!!,english,ahora feliz mucha salud para tod
1,2,2,message.y,136530,psoriasis,105226382844778_320199574680790,2012-01-01 00:29:53,Ms piel y menos placas!!!!!!,english,piel menos placas
2,6,6,message.y,136534,psoriasis,105226382844778_320199574680790,2012-01-01 00:41:29,"muchas gracias, adios picores y hola nueva pie...",english,muchas gracias adios picores hola nueva piel


116141
